<a href="https://colab.research.google.com/github/richardparik/Projects/blob/master/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brain Tumor

Dataset k projektu
https://www.kaggle.com/datasets/denizkavi1/brain-tumor?resource=download

1. 1 Meningioma samples (708 files)
2. 2 Glioma samples (1426 files)
3. 3 Pituitary tumor samples (930 files)



In [ ]:
zmena zmeny